# Protectora Barcelona

Find a list of adoptable dogs od the Protectora de Barcelona https://www.protectorabcn.es/

<img src='../img/goliath.jpg' width='400' height='400'/>

## 0. Import libraries

In [1]:
# Web-scraping
import requests 
from bs4 import BeautifulSoup as bs
import os
from urllib.parse import urlparse
import re
import time

# Numpy y pandas
import numpy as np
import pandas as pd

# Options for DataFrame visualization:
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 40)
# Option to show not truncated cells in pandas
pd.set_option('display.max_colwidth', -1)

## 1. Download and display the content of robots.txt

In [2]:
def robot_txt():
    response = requests.get('https://www.protectorabcn.es/robots.txt')
    test = response.text
    print('robots.txt for https://www.protectorabcn.es/')
    print('============================================')
    result_data_set = {'DISALLOWED':[], 'ALLOWED':[]}

    for line in test.split('\n'):
        if line.startswith('Allow'):    # this is for allowed url
            result_data_set['ALLOWED'].append(line.split(': ')[1].split(' ')[0])    
        elif line.startswith('Disallow'):    # this is for disallowed url
            result_data_set['DISALLOWED'].append(line.split(': ')[1].split(' ')[0])    

    return result_data_set
robot_txt()

robots.txt for https://www.protectorabcn.es/


{'DISALLOWED': ['/*?orderby=',
  '/*?orderway=',
  '/*?tag=',
  '/*?id_currency=',
  '/*?search_query=',
  '/*?back=',
  '/*?n=',
  '/*&orderby=',
  '/*&orderway=',
  '/*&tag=',
  '/*&id_currency=',
  '/*&search_query=',
  '/*&back=',
  '/*&n=',
  '/*controller=addresses',
  '/*controller=address',
  '/*controller=authentication',
  '/*controller=cart',
  '/*controller=discount',
  '/*controller=footer',
  '/*controller=get-file',
  '/*controller=header',
  '/*controller=history',
  '/*controller=identity',
  '/*controller=images.inc',
  '/*controller=init',
  '/*controller=my-account',
  '/*controller=order',
  '/*controller=order-opc',
  '/*controller=order-slip',
  '/*controller=order-detail',
  '/*controller=order-follow',
  '/*controller=order-return',
  '/*controller=order-confirmation',
  '/*controller=pagination',
  '/*controller=password',
  '/*controller=pdf-invoice',
  '/*controller=pdf-order-return',
  '/*controller=pdf-order-slip',
  '/*controller=product-sort',
  '/*contr

The pages I want to scrape (https://www.protectorabcn.es/5-perros-en-adopcion) have no restrictions

## 2. Create a random User Agent generator

In [3]:
def get_random_ua():
    random_ua = ''
    ua_file = 'ua_file.txt'
    try:
        with open(ua_file) as f:
            lines = f.readlines()
        if len(lines) > 0:
            # random.RandomState exposes a number of methods for generating random numbers drawn from a variety of probability distributions
            prng = np.random.RandomState()
            index = prng.permutation(len(lines) - 1)
            idx = np.asarray(index, dtype=np.integer)[0]
            random_ua = lines[int(idx)]
    except Exception as ex:
        print('Exception in random_ua')
        print(str(ex))
    finally:
        return random_ua

# 3. Web scraping

## 3.1 Main page dogs' adoptions

In [4]:
def total_adoptable_dogs():   
    '''
    Find the number of adoptable dogs availables
    '''
    url = f'https://www.protectorabcn.es/5-perros-en-adopcion'
    user_agent = get_random_ua().replace('\n','')
    headers = {'user_agent':user_agent}
    
    html = requests.get(url, headers).text
    soup = bs(html, 'lxml')

    tot_dogs = soup.find('span', class_='heading-counter').text

    for dogs in tot_dogs.split():
        if dogs.isdigit():
            num_dogs = int(dogs)
            return num_dogs

In [5]:
print (f'At the moment, there are {total_adoptable_dogs()} dogs available for adoption')

At the moment, there are 43 dogs available for adoption


In [6]:
def list_adoptable_dogs():
    '''
    Find list of the adoptable dogs available with description and fotos
    '''
    
    url = f'https://www.protectorabcn.es/5-perros-en-adopcion?id_category=5&n={total_adoptable_dogs()}'

    user_agent = get_random_ua()
    headers = {'user_agent':user_agent}
    
    html = requests.get(url, headers).text
    soup = bs(html, 'lxml')
    
    dogs = soup.find_all('div', class_='product-container')
    
    adoptable_dogs = []

    for dog in dogs:
        adoptable_dogs.append ({
        'Nombre': dog.find('h5').find('a')['title'],
        'Caracteristicas': dog.find('span', class_='product-reference').text,
        'Foto': dog.find('img').get('src'),
        # using the libraries 'urllib.parse' and 'os'
        'URL' : (os.path.basename(urlparse(dog.find('a')['href']).path).replace('.html','').split("-")[1]),
        'Codigo': (''.join([dog for dog in dog.find('a')['href'] if dog.isdigit()]))
    })
        
    adoptable_dogs_df = pd.DataFrame(adoptable_dogs)
    
    return adoptable_dogs_df

In [7]:
# Data extract from the main page
df_main_page = list_adoptable_dogs()
df_main_page.head()

,Nombre,Caracteristicas,Foto,URL,Codigo
0,Gina,Curiosa y elegante,https://server03.protectorabcn.es/8935-home_default/gina.jpg,gina,39
1,Horus,Una familia con dedicación,https://server02.protectorabcn.es/8943-home_default/base-perros.jpg,base,997
2,Muñeca,Preparada para volver a empezar,https://server01.protectorabcn.es/6424-home_default/muneca.jpg,muneca,1291
3,Goliath,Necesita un hogar urgente,https://server03.protectorabcn.es/8344-home_default/goliath.jpg,goliath,1481
4,Bruno,Una persona que lo entienda,https://server03.protectorabcn.es/8682-home_default/bruno.jpg,bruno,1054


## 3.2 Scrape single pages of the adopbtable dogs 

In [8]:
def dog_scraper(url):
    '''
    Scraper for the single pages of the dogs
    '''
    user_agent = get_random_ua()
    headers = {'user_agent':user_agent}
    html = requests.get(url, headers).text
    soup = bs(html, 'lxml')
    
    # create database with information about: name, past and future of the dogs
    general_info = []
    name = soup.find('div', class_='product-title').find('h1').text
    description = (soup.find('div', class_='rte align_justify').text.replace('\xa0',''))
    general_info.append((name, description))

    df1 =pd.DataFrame(general_info, columns =['Nombre', 'Descripción'])
#     df1
    
    # create database with data from the table
    tables = soup.find_all('table')
    table = tables[0]
    tab_data = [[cell.text for cell in row.find_all(['th','td'])]
                            for row in table.find_all('tr')]

    df2 = pd.DataFrame(tab_data).T
    df2.columns = df2.iloc[0,:]
    df2.drop(index=0,inplace=True)
    df2.reset_index(drop = True, inplace = True)
#     df2
    
    # join two dataframes 
    try:
        df_dogs2 = pd.concat([df1,df2], axis=1, sort=False)
        df_dogs2 = df_dogs2[['Nombre', 'Animal', 'Sexo', 'Medidas', 'Edad', 'Color', 'Raza', 'Nacimiento', 'Entrada', 'Descripción', 'Estado']]
    # some dataframes doesn't have the column 'Color'
    except:
        df2['Color']='No disponible'
        df_dogs2 = pd.concat([df1,df2], axis=1, sort=False)
        df_dogs2 = df_dogs2[['Nombre', 'Animal', 'Sexo', 'Medidas', 'Edad', 'Color', 'Raza', 'Nacimiento', 'Entrada', 'Descripción', 'Estado']]

    return df_dogs2

Test the first page of adoptable dogs 

In [9]:
url_nerea = 'https://www.protectorabcn.es/perros-en-adopcion/78-nerea.html'
dog_scraper(url_nerea)

,Nombre,Animal,Sexo,Medidas,Edad,Color,Raza,Nacimiento,Entrada,Descripción,Estado
0,Nerea,Perros,Hembra,Mediano,Adulto,Marrón claro,Mestizo,2010,02-2014,"Nerea fue rescatada de un síndrome de Noé hace ya 5 años y desde entonces ha estado en la Lliga. Era un ambiente muy complejo que la condicionó, pero desde que llegó al refugio ha mejorado mucho. Aún le cuesta mostrarse afectuosa con las personas, así que buscamos para ella una familia que tenga paciencia y comprensión con ella. Nerea tiene 8 años y necesita que por fin una familia se fije en ella y le dé el hogar que necesita.Debido a su pasado, Nerea necesita estar acompañada, con lo cual es preferible una familia con algún otro perro, o en la que Nerea esté con personas todo el tiempo. También es incompatible con gatos. Por otro lado es una perrita a la que le encanta pasear y tomar el sol. Esperamos que su oportunidad de ser feliz llegue pronto.Si estás interesado en conocerla puedes enviarnos un email a info@protectorabcn.es",Adoptado


In [10]:
# example of dogs for which the color is not specified
url_marley = 'https://www.protectorabcn.es/perros-en-adopcion/1849-marley.html'
dog_scraper(url_marley)

,Nombre,Animal,Sexo,Medidas,Edad,Color,Raza,Nacimiento,Entrada,Descripción,Estado
0,Marley,Perros,Macho,Grande,Adulto,No disponible,Pastor belga malinois,10.2018,07.2020,"Marley ha tenido un pasado muy desafortunado y ha llegado a la Lliga en busca de un mejor futuro. Su antiguo propietario no lo atendía correctamente y por este motivo necesita aprender a pasear por la calle con tranquilidad. Marley es un Malinois joven y con mucha energía. Por el tipo de raza que es, no es adecuado para la mayoría de familias que buscan un perro como animal de compañía, los Malinois son perros con un nivel de actividad muy alto y con potencia, por lo que vamos a valorar únicamente las candidaturas que veamos que puedan ser apropiadas para él. Marley es un perro sociable con personas y con perros.Si estás interesado en conocerlo puedes enviarnos un email a info@protectorabcn.es",En Adopción


## 3.3 Find the URLS for all the adoptable dogs

Extract all the codes from the main page to create all the URL for the single dogs. 
Example: https://www.protectorabcn.es/perros-en-adopcion/78-nerea.html

In [11]:
adoptable_dogs = df_main_page["Nombre"].tolist()
URL_dogs = df_main_page["URL"].tolist()
code_dogs = df_main_page["Codigo"].tolist()

df_codes = pd.DataFrame((adoptable_dogs, URL_dogs, code_dogs))
df_codes = df_codes.rename(index={0: 'Nombre', 1: 'URL', 2: 'Codigo'})
df_codes

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,...,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42
Nombre,Gina,Horus,Muñeca,Goliath,Bruno,Skaner,Riki,Pepa,Tay,Basil,Tona,Toby,Neo,Simba,Tizón,Thor,Blai,Luna,Osiris,Maggie,...,Olivia,Cotó,Rosalia,Marley,Yara,Pongo,Lisa,Duna,Tronca,Akami,Otta,Leia,Coral,Harry,Elvira,Wendy,Pinto,Calipso,Joao,Momo
URL,gina,base,muneca,goliath,bruno,skaner,riki,pepa,tay,basil,tona,toby,neo,simba,tizon,thor,blai,luna,osiris,maggie,...,olivia,coto,rosalia,marley,yara,base,lisa,duna,tronca,akami,otta,leia,coral,harry,elvira,wendy,pinto,calipso,base,momo
Codigo,39,997,1291,1481,1054,1511,1531,1592,1353,1632,1120,1467,1744,1771,1772,1784,1825,1831,1653,1840,...,1846,1847,1848,1849,1852,1633,1865,1866,1867,1868,1869,1870,1871,1872,1873,1875,1876,1877,1677,1289


In [12]:
def all_URLs(url):
    '''
    Scraper for the single pages of the dogs
    '''
    
    all_urls = []   
    for name, code in zip(URL_dogs, code_dogs):

        #Build URL for each dog
        all_URLs = base_url + code + '-' + name +'.html'
        
        #Pause
        time.sleep(1)
        
        all_urls.append(all_URLs)
        
    return all_urls

Creation of the URLs for every dog

In [13]:
base_url = 'https://www.protectorabcn.es/perros-en-adopcion/'
all_urls = all_URLs(base_url)
all_urls

['https://www.protectorabcn.es/perros-en-adopcion/39-gina.html',
 'https://www.protectorabcn.es/perros-en-adopcion/997-base.html',
 'https://www.protectorabcn.es/perros-en-adopcion/1291-muneca.html',
 'https://www.protectorabcn.es/perros-en-adopcion/1481-goliath.html',
 'https://www.protectorabcn.es/perros-en-adopcion/1054-bruno.html',
 'https://www.protectorabcn.es/perros-en-adopcion/1511-skaner.html',
 'https://www.protectorabcn.es/perros-en-adopcion/1531-riki.html',
 'https://www.protectorabcn.es/perros-en-adopcion/1592-pepa.html',
 'https://www.protectorabcn.es/perros-en-adopcion/1353-tay.html',
 'https://www.protectorabcn.es/perros-en-adopcion/1632-basil.html',
 'https://www.protectorabcn.es/perros-en-adopcion/1120-tona.html',
 'https://www.protectorabcn.es/perros-en-adopcion/1467-toby.html',
 'https://www.protectorabcn.es/perros-en-adopcion/1744-neo.html',
 'https://www.protectorabcn.es/perros-en-adopcion/1771-simba.html',
 'https://www.protectorabcn.es/perros-en-adopcion/1772-ti

Test all URLs

In [14]:
# In order to concatenate dataframes, I have to create a list of all the dogs
adoptable_dogs = []
for i in all_urls:
    adoptable_dogs.append(dog_scraper(i))  
#     print(adoptable_dogs)

In [15]:
list_dogs = []
for i in adoptable_dogs:
    list_dogs.append(i)
# print(list_dogs)

In [16]:
# Data extract from the pages of every single dog
df_single_pages = pd.concat(list_dogs, ignore_index=True)
df_single_pages.head()

,Nombre,Animal,Sexo,Medidas,Edad,Color,Raza,Nacimiento,Entrada,Descripción,Estado
0,Gina,Perros,Hembra,Grande,Adulto,Blanco,Mestizo,2010,02.2014,"Si tuviéramos que describir a Gina con una sola palabra sería: “cautivadora”. Así es, Gina es una perra con mucho encanto, si no lo crees pregúntale a los voluntarios, resulta muy bonito ver la estima que le tienen. Gina hace que todo sea especial, desde su forma de comer o sus paseos, hasta la efusividad con la que te recibe, todo en ella está repleto de detalles que la hacen única. No ha tenido un pasado fácil, Gina sufrió malos tratos por parte del que era su propietario, y vivió en un entorno muy desequilibrado. Cuando llegó al refugio en 2014 Gina parecía una cabra desbocada, no entendía absolutamente nada, y costó muchos meses de trabajo enseñarle y reconducir sus conductas asalvajadas. Tras estos años con nosotros Gina ha dado un cambio espectacular, aquella cabra loca ya forma parte del recuerdo de voluntarios y trabajadores, ahora Gina es toda una señorita que busca una familia que le aporte estabilidad y la ayude a seguir el camino de mejora que ha tomado. Buscamos para ella un/a adoptante que preferiblemente tenga otro perro o bien una familia en la que estuviera acompañada todo el día, ya que siempre ha convivido con perros y es posible que no lleve bien quedarse sola. A Gina le quedan muchos años de vida por delante y todavía le falta descubrir lo que es un hogar, si no puedes adoptar, puedes ayudar compartiendo su caso para que por fin encuentre su lugar.Si estás interesado en conocerla envíanos un email a info@protectorabcn.es",En Adopción
1,Horus,Perros,Macho,Grande,Joven,Marron oscuro,Mestizo de Pastor Alemán,03.2016,09.2017,"Horus es este cruce de pastor alemán, de 3 años de edad. A Horus le encanta jugar y dar paseos tranquilos, y es muy cariñoso con las personas que conoce. Por otro lado, es desconfiado con desconocidos y necesita de una familia que tenga paciencia para conocerlo y para ayudarlo en su adaptación. Por otro lado, Horus es muy noble y besucón con sus personas.Esperamos que llegue esa familia que le dé el apoyo que necesita para sentirse seguro y por fin le dé el hogar que tanto merece. Con Horus descubrirán un perro maravilloso que, aunque necesita paciencia para adaptarse, corresponde a sus personas con una fidelidad absoluta y devoción por ellos.Si estás interesado en conocerlo puedes enviarnos un email a info@protectorabcn.es",En Adopción
2,Muñeca,Perros,Hembra,Mediano,Adulto,Negro,Mestizo,02.2012,07.2018,"Muñeca es esta perrita cariñosa y buena nacida en 2012, y con mucha vida por delante, que espera que por fin alguien se fije en ella. Llegó hace unos meses a nuestro refugio procedente de otro refugio que tuvo que cerrar. Muñeca es una perrita vital y activa, pero que a la vez sabe estar tranquila. Es efusiva y disfruta mucho de los paseos, pero hay que invertir tiempo en conocerla porque, igual que las personas, tiene sus manías. Por otro lado, es una perrita que cuando la conoces, te enamora, y creará un fuerte vínculo con esas personas que decidan ser su familia para siempre.Si estás interesado en conocerla puedes enviarnos un email a info@protectorabcn.es",En Adopción
3,Goliath,Perros,Macho,Grande,Joven,Gris,Fila Brasileiro,03.2018,02.2019,"Buscamos una acogida indefinida para nuestro Goliath. No tiene ni 2 añitos pero, debido a su tamaño, tiene una artrosis avanzada y el frío, la humedad y el gran número de escaleras, hace que el refugio no sea un buen lugar para él.Buscamos para él un hogar sin escaleras y sin niños. Puesto que de entrada es desconfiado con los desconocidos, necesitamos que la familia cree un vínculo con él antes de llevárselo, ya que una vez creadoes un perrito muyalegre, cariñoso, juguetón y extremadamente fiel.Si alguno podéis tenderle una patita a Goliath o conocéis a alguien que pueda hacerlo, por favor, contactadnos.",Urgente
4,Bruno,Perros,Macho,Pequeño,Adulto,Beig,Mestizo de Terrier,10.2011,01.2018,Bruno es este pr

## 4. Union of DF

Merge the dataframes 

In [17]:
# Main page
df_main_page.head()

,Nombre,Caracteristicas,Foto,URL,Codigo
0,Gina,Curiosa y elegante,https://server03.protectorabcn.es/8935-home_default/gina.jpg,gina,39
1,Horus,Una familia con dedicación,https://server02.protectorabcn.es/8943-home_default/base-perros.jpg,base,997
2,Muñeca,Preparada para volver a empezar,https://server01.protectorabcn.es/6424-home_default/muneca.jpg,muneca,1291
3,Goliath,Necesita un hogar urgente,https://server03.protectorabcn.es/8344-home_default/goliath.jpg,goliath,1481
4,Bruno,Una persona que lo entienda,https://server03.protectorabcn.es/8682-home_default/bruno.jpg,bruno,1054


In [18]:
# Dogs' details page
df_single_pages.head()

,Nombre,Animal,Sexo,Medidas,Edad,Color,Raza,Nacimiento,Entrada,Descripción,Estado
0,Gina,Perros,Hembra,Grande,Adulto,Blanco,Mestizo,2010,02.2014,"Si tuviéramos que describir a Gina con una sola palabra sería: “cautivadora”. Así es, Gina es una perra con mucho encanto, si no lo crees pregúntale a los voluntarios, resulta muy bonito ver la estima que le tienen. Gina hace que todo sea especial, desde su forma de comer o sus paseos, hasta la efusividad con la que te recibe, todo en ella está repleto de detalles que la hacen única. No ha tenido un pasado fácil, Gina sufrió malos tratos por parte del que era su propietario, y vivió en un entorno muy desequilibrado. Cuando llegó al refugio en 2014 Gina parecía una cabra desbocada, no entendía absolutamente nada, y costó muchos meses de trabajo enseñarle y reconducir sus conductas asalvajadas. Tras estos años con nosotros Gina ha dado un cambio espectacular, aquella cabra loca ya forma parte del recuerdo de voluntarios y trabajadores, ahora Gina es toda una señorita que busca una familia que le aporte estabilidad y la ayude a seguir el camino de mejora que ha tomado. Buscamos para ella un/a adoptante que preferiblemente tenga otro perro o bien una familia en la que estuviera acompañada todo el día, ya que siempre ha convivido con perros y es posible que no lleve bien quedarse sola. A Gina le quedan muchos años de vida por delante y todavía le falta descubrir lo que es un hogar, si no puedes adoptar, puedes ayudar compartiendo su caso para que por fin encuentre su lugar.Si estás interesado en conocerla envíanos un email a info@protectorabcn.es",En Adopción
1,Horus,Perros,Macho,Grande,Joven,Marron oscuro,Mestizo de Pastor Alemán,03.2016,09.2017,"Horus es este cruce de pastor alemán, de 3 años de edad. A Horus le encanta jugar y dar paseos tranquilos, y es muy cariñoso con las personas que conoce. Por otro lado, es desconfiado con desconocidos y necesita de una familia que tenga paciencia para conocerlo y para ayudarlo en su adaptación. Por otro lado, Horus es muy noble y besucón con sus personas.Esperamos que llegue esa familia que le dé el apoyo que necesita para sentirse seguro y por fin le dé el hogar que tanto merece. Con Horus descubrirán un perro maravilloso que, aunque necesita paciencia para adaptarse, corresponde a sus personas con una fidelidad absoluta y devoción por ellos.Si estás interesado en conocerlo puedes enviarnos un email a info@protectorabcn.es",En Adopción
2,Muñeca,Perros,Hembra,Mediano,Adulto,Negro,Mestizo,02.2012,07.2018,"Muñeca es esta perrita cariñosa y buena nacida en 2012, y con mucha vida por delante, que espera que por fin alguien se fije en ella. Llegó hace unos meses a nuestro refugio procedente de otro refugio que tuvo que cerrar. Muñeca es una perrita vital y activa, pero que a la vez sabe estar tranquila. Es efusiva y disfruta mucho de los paseos, pero hay que invertir tiempo en conocerla porque, igual que las personas, tiene sus manías. Por otro lado, es una perrita que cuando la conoces, te enamora, y creará un fuerte vínculo con esas personas que decidan ser su familia para siempre.Si estás interesado en conocerla puedes enviarnos un email a info@protectorabcn.es",En Adopción
3,Goliath,Perros,Macho,Grande,Joven,Gris,Fila Brasileiro,03.2018,02.2019,"Buscamos una acogida indefinida para nuestro Goliath. No tiene ni 2 añitos pero, debido a su tamaño, tiene una artrosis avanzada y el frío, la humedad y el gran número de escaleras, hace que el refugio no sea un buen lugar para él.Buscamos para él un hogar sin escaleras y sin niños. Puesto que de entrada es desconfiado con los desconocidos, necesitamos que la familia cree un vínculo con él antes de llevárselo, ya que una vez creadoes un perrito muyalegre, cariñoso, juguetón y extremadamente fiel.Si alguno podéis tenderle una patita a Goliath o conocéis a alguien que pueda hacerlo, por favor, contactadnos.",Urgente
4,Bruno,Perros,Macho,Pequeño,Adulto,Beig,Mestizo de Terrier,10.2011,01.2018,Bruno es este pr

In [19]:
final_df = df_main_page.merge(df_single_pages, left_on='Nombre', right_on='Nombre')
final_df.head(2)

,Nombre,Caracteristicas,Foto,URL,Codigo,Animal,Sexo,Medidas,Edad,Color,Raza,Nacimiento,Entrada,Descripción,Estado
0,Gina,Curiosa y elegante,https://server03.protectorabcn.es/8935-home_default/gina.jpg,gina,39,Perros,Hembra,Grande,Adulto,Blanco,Mestizo,2010,02.2014,"Si tuviéramos que describir a Gina con una sola palabra sería: “cautivadora”. Así es, Gina es una perra con mucho encanto, si no lo crees pregúntale a los voluntarios, resulta muy bonito ver la estima que le tienen. Gina hace que todo sea especial, desde su forma de comer o sus paseos, hasta la efusividad con la que te recibe, todo en ella está repleto de detalles que la hacen única. No ha tenido un pasado fácil, Gina sufrió malos tratos por parte del que era su propietario, y vivió en un entorno muy desequilibrado. Cuando llegó al refugio en 2014 Gina parecía una cabra desbocada, no entendía absolutamente nada, y costó muchos meses de trabajo enseñarle y reconducir sus conductas asalvajadas. Tras estos años con nosotros Gina ha dado un cambio espectacular, aquella cabra loca ya forma parte del recuerdo de voluntarios y trabajadores, ahora Gina es toda una señorita que busca una familia que le aporte estabilidad y la ayude a seguir el camino de mejora que ha tomado. Buscamos para ella un/a adoptante que preferiblemente tenga otro perro o bien una familia en la que estuviera acompañada todo el día, ya que siempre ha convivido con perros y es posible que no lleve bien quedarse sola. A Gina le quedan muchos años de vida por delante y todavía le falta descubrir lo que es un hogar, si no puedes adoptar, puedes ayudar compartiendo su caso para que por fin encuentre su lugar.Si estás interesado en conocerla envíanos un email a info@protectorabcn.es",En Adopción
1,Horus,Una familia con dedicación,https://server02.protectorabcn.es/8943-home_default/base-perros.jpg,base,997,Perros,Macho,Grande,Joven,Marron oscuro,Mestizo de Pastor Alemán,03.2016,09.2017,"Horus es este cruce de pastor alemán, de 3 años de edad. A Horus le encanta jugar y dar paseos tranquilos, y es muy cariñoso con las personas que conoce. Por otro lado, es desconfiado con desconocidos y necesita de una familia que tenga paciencia para conocerlo y para ayudarlo en su adaptación. Por otro lado, Horus es muy noble y besucón con sus personas.Esperamos que llegue esa familia que le dé el apoyo que necesita para sentirse seguro y por fin le dé el hogar que tanto merece. Con Horus descubrirán un perro maravilloso que, aunque necesita paciencia para adaptarse, corresponde a sus personas con una fidelidad absoluta y devoción por ellos.Si estás interesado en conocerlo puedes enviarnos un email a info@protectorabcn.es",En Adopción


## 5. Date format modifications

String formatting

In [20]:
final_df['Entrada'] = final_df['Entrada'].str.replace('.','/').str.replace('-','/').replace('.0000','')

In [21]:
final_df['Nacimiento'] = final_df['Nacimiento'].str.replace('.','/').replace('.0000','')

#### · Años de perrera

In [22]:
# Conversion of the dates from string to datetime format
final_df['Años perrera'] = round((pd.to_datetime(final_df['Entrada'])-pd.to_datetime(final_df['Nacimiento'])).dt.days / 365.25, 1)

In [23]:
# String formatting
final_df['Años perrera'] = final_df['Años perrera'].apply(lambda x: str(x)+' years')
# final_df['Años perrera']

In [24]:
dogs_df = final_df[['Nombre','Caracteristicas','Foto','URL','Codigo','Animal','Sexo','Medidas','Edad','Color','Raza','Nacimiento','Entrada','Años perrera','Descripción','Estado']]
dogs_df.head(2)

,Nombre,Caracteristicas,Foto,URL,Codigo,Animal,Sexo,Medidas,Edad,Color,Raza,Nacimiento,Entrada,Años perrera,Descripción,Estado
0,Gina,Curiosa y elegante,https://server03.protectorabcn.es/8935-home_default/gina.jpg,gina,39,Perros,Hembra,Grande,Adulto,Blanco,Mestizo,2010,02/2014,4.1 years,"Si tuviéramos que describir a Gina con una sola palabra sería: “cautivadora”. Así es, Gina es una perra con mucho encanto, si no lo crees pregúntale a los voluntarios, resulta muy bonito ver la estima que le tienen. Gina hace que todo sea especial, desde su forma de comer o sus paseos, hasta la efusividad con la que te recibe, todo en ella está repleto de detalles que la hacen única. No ha tenido un pasado fácil, Gina sufrió malos tratos por parte del que era su propietario, y vivió en un entorno muy desequilibrado. Cuando llegó al refugio en 2014 Gina parecía una cabra desbocada, no entendía absolutamente nada, y costó muchos meses de trabajo enseñarle y reconducir sus conductas asalvajadas. Tras estos años con nosotros Gina ha dado un cambio espectacular, aquella cabra loca ya forma parte del recuerdo de voluntarios y trabajadores, ahora Gina es toda una señorita que busca una familia que le aporte estabilidad y la ayude a seguir el camino de mejora que ha tomado. Buscamos para ella un/a adoptante que preferiblemente tenga otro perro o bien una familia en la que estuviera acompañada todo el día, ya que siempre ha convivido con perros y es posible que no lleve bien quedarse sola. A Gina le quedan muchos años de vida por delante y todavía le falta descubrir lo que es un hogar, si no puedes adoptar, puedes ayudar compartiendo su caso para que por fin encuentre su lugar.Si estás interesado en conocerla envíanos un email a info@protectorabcn.es",En Adopción
1,Horus,Una familia con dedicación,https://server02.protectorabcn.es/8943-home_default/base-perros.jpg,base,997,Perros,Macho,Grande,Joven,Marron oscuro,Mestizo de Pastor Alemán,03/2016,09/2017,1.5 years,"Horus es este cruce de pastor alemán, de 3 años de edad. A Horus le encanta jugar y dar paseos tranquilos, y es muy cariñoso con las personas que conoce. Por otro lado, es desconfiado con desconocidos y necesita de una familia que tenga paciencia para conocerlo y para ayudarlo en su adaptación. Por otro lado, Horus es muy noble y besucón con sus personas.Esperamos que llegue esa familia que le dé el apoyo que necesita para sentirse seguro y por fin le dé el hogar que tanto merece. Con Horus descubrirán un perro maravilloso que, aunque necesita paciencia para adaptarse, corresponde a sus personas con una fidelidad absoluta y devoción por ellos.Si estás interesado en conocerlo puedes enviarnos un email a info@protectorabcn.es",En Adopción


## 6. Export final DF

Export final dataframe in csv

In [25]:
adoptable_dogs = 'adoptable_dogs '+time.strftime('%d-%m-%Y_%H-%M')+'.csv'
adoptable_dogs

'adoptable_dogs 25-09-2020_11-53.csv'

In [27]:
final_df.to_csv('../data/'+adoptable_dogs, sep=",", index=False)

In [28]:
# Share data between Jupyter Notebooks
%store adoptable_dogs

Stored 'adoptable_dogs' (str)
